# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# 기상청 API 연동 테스트

이 노트북에서는 다음 작업을 수행합니다:  
1. 기상청 API 연동을 위한 필요한 라이브러리 임포트  
2. API 키 설정 및 확인  
3. 데이터 수집 테스트  

In [4]:
# 1. 필요한 라이브러리 임포트
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

False

In [7]:
# 2. API 키 확인
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')
if not WEATHER_API_KEY:
    raise ValueError("WEATHER_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")
print("API 키가 정상적으로 설정되었습니다.")


API 키가 정상적으로 설정되었습니다.


In [14]:
# 3.데이터 수집 테스트
url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
params = {
    "serviceKey": WEATHER_API_KEY, # 서비스 키
    "pageNo": "1", # 페이지 번호
    "numOfRows": "10", # 한 페이지 결과 수
    "dataType": "JSON", # 응답자료형식
    "dataCd": "ASOS", # 자료코드
    "dateCd": "DAY", # 날짜코드
    "startDt": "20000101", # 시작일
    "endDt": "20220101", # 종료일
    "stnIds": "108", # 종관기상관측 지점 번호
}

# 요청 보내기
res = requests.get(url, params=params)
res.raise_for_status()  # 응답 실패시 예외 발생

# JSON 파싱
response = res.json()

# 응답 데이터 → DataFrame
try:
    items = response['response']['body']['items']['item']
    df = pd.DataFrame(items)
    print("✅ 수집된 데이터:")
    display(df.head())
except KeyError as e:
    print("⚠️ 데이터 파싱 실패:", e)
    print("응답 내용:\n", response)

✅ 수집된 데이터:


,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,108,서울,2000-01-01,5.5,1.8,0615,9.9,1441,,,...,3.5,6.7,10.0,14.6,17.5,,0.9,4.0,-{박무}-{박무}{강도0}0300-{박무}{강도0}0600-{박무}{강도0}090...,
1,108,서울,2000-01-02,4.2,-0.9,2359,6.9,0027,,,...,3.6,6.7,9.7,14.5,17.5,,0.8,2.0,{비}0445-{비}{강도0}0600-{비}{강도0}0900-1135. {뇌전}{...,
2,108,서울,2000-01-03,-2.2,-4.6,0829,0.1,1425,,,...,3.3,6.2,9.5,14.5,17.8,,0.8,,{박무}0320-{박무}{강도0}0600-0740.,
3,108,서울,2000-01-04,0.3,-4.3,0535,4.3,1510,,,...,3.3,6.3,9.5,14.5,17.3,,0.7,3.7,{연무}1035-{연무}{강도0}1200-{연무}{강도0}1500-1750.,
4,108,서울,2000-01-05,2.8,0.1,0736,4.6,2311,,,...,3.3,6.0,9.5,14.5,17.5,,1.7,24.5,{눈}0610-{눈}{강도0}0900-{진눈깨비}0920-{비}0950-{비}{강도...,


| 항목명 (영문명(한글명))                  | 설명                           |
|------------------------------------------|--------------------------------|
| resultCode(결과코드)                     | 결과코드                       |
| resultMsg(결과메시지)                    | 결과메시지                     |
| numOfRows(한 페이지 결과 수)             | 한 페이지 결과 수              |
| pageNo(페이지 번호)                      | 페이지 번호                    |
| totalCount(전체 결과 수)                 | 전체 결과 수                   |
| dataType(데이터 타입)                    | 응답자료형식 (XML/JSON)        |
| tm(시간)                                 | 일시                           |
| stnId(지점 번호)                         | 종관기상관측 지점 번호         |
| avgM15Te(1.5m 지중온도)                  | 1.5m 지중온도                  |
| n99Rn(9-9강수)                            | 9-9강수                        |
| minPs(최저 해면기압)                     | 최저 해면기압                  |
| avgRhm(평균 상대습도)                    | 평균 상대습도                  |
| minRhmHrmt(평균 상대습도 시각)           | 평균 상대습도 시각             |
| maxInsWsWd(최대 순간 풍속 풍향)          | 최대 순간 풍속 풍향            |
| avgTs(평균 지면온도)                     | 평균 지면온도                  |
| max_ins_ws_hrmt(최대 순간풍속 시각)      | 최대 순간풍속 시각             |
| ddMesHrmt(일 최심적설 시각)              | 일 최심적설 시각               |
| maxPsHrmt(최고 해면기압 시각)            | 최고 해면기압 시각             |
| avgPv(평균 증기압)                       | 평균 증기압                    |
| minRhm(최소 상대습도)                    | 최소 상대습도                  |
| sumSsHr(합계 일조 시간)                  | 합계 일조 시간                 |
| ssDur(가조시간)                          | 가조시간                       |
| avgPs(평균 해면기압)                     | 평균 해면기압                  |
| maxWs(최대 풍속)                         | 최대 풍속                      |
| avgCm5Te(평균 5cm 지중온도)              | 평균 5cm 지중온도              |
| minTg(최저 초상온도)                     | 최저 초상온도                  |
| maxWsWd(최대 풍속 풍향)                 | 최대 풍속 풍향                 |
| sumSmlEv(합계 소형증발량)                | 합계 소형증발량                |
| avgTca(평균 전운량)                      | 평균 전운량                    |
| hr1MaxIcsr(1시간 최다 일사량)            | 1시간 최다 일사량              |
| avgTd(평균 이슬점온도)                   | 평균 이슬점온도                |
| maxPs(최고 해면기압)                     | 최고 해면기압                  |
| avgCm20Te(평균 20cm 지중온도)            | 평균 20cm 지중온도             |
| ddMes(일 최심적설)                       | 일 최심적설                    |
| minTa(최저 기온)                         | 최저 기온                      |
| minPsHrmt(최저 해면기압 시각)            | 최저 해면기압 시각             |
| avgM50Te(5.0m 지중온도)                  | 5.0m 지중온도                  |
| maxTa(최고 기온)                         | 최고 기온                      |
| hr24SumRws(풍정합)                       | 풍정합                         |
| avgM30Te(3.0m 지중온도)                  | 3.0m 지중온도                  |
| avgCm10Te(평균 10cm 지중온도)            | 평균 10cm 지중온도             |
| avgM05Te(0.5m 지중온도)                  | 0.5m 지중온도                  |
| hr1MaxIcsrHrmt(1시간 최다 일사량 시각)   | 1시간 최다 일사량 시각         |
| maxInsWs(최대 순간풍속)                  | 최대 순간풍속                  |
| avgCm30Te(평균 30cm 지중온도)            | 평균 30cm 지중온도             |
| avgM10Te(1.0m 지중온도)                  | 1.0m 지중온도                  |
| sumGsr(합계 일사)                        | 합계 일사                      |
| maxWsHrmt(최대 풍속 시각)                | 최대 풍속 시각                 |
| avgPa(평균 현지기압)                     | 평균 현지기압                  |
| avgWs(평균 풍속)                         | 평균 풍속                      |
| iscs(일기현상)                           | 일기현상                       |
| sumFogDur(안개 계속 시간)                | 안개 계속 시간                 |
| sumLrgEv(합계 대형증발량)                | 합계 대형증발량                |
| sumDpthFhsc(합계 3시간 신적설)           | 합계 3시간 신적설              |
| ddMefs(일 최심신적설)                    | 일 최심신적설                  |
| ddMefsHrmt(일 최심신적설 시각)           | 일 최심신적설 시각             |
| sumRn(일강수량)                          | 일강수량                       |
| hr1MaxRnHrmt(1시간 최다 강수량 시각)     | 1시간 최다 강수량 시각         |
| hr1MaxRn(1시간 최다강수량)               | 1시간 최다강수량               |
| mi10MaxRnHrmt(10분 최다강수량 시각)      | 10분 최다강수량 시각           |
| mi10_max_rn(10분 최다 강수량)            | 10분 최다 강수량               |
| avgTa(평균 기온)                         | 평균 기온                      |
| minTaHrmt(최저 기온 시각)                | 최저 기온 시각                 |
| maxTaHrmt(최고 기온 시각)                | 최고 기온 시각                 |
| maxWd(최대 풍향)                         | 최대 풍향                      |
| avgLmac(평균 중하층운량)                | 평균 중하층운량                |

# Naver Cloud Storage 연동 테스트
1. 네이버 클라우드 오브젝트 스토리지 접속을 위한 설정 및 클라이언트 생성  
2. pandas DataFrame을 CSV 형태로 저장하여 스토리지에 업로드  
3. 스토리지에 저장된 CSV 파일을 불러와 DataFrame으로 로드  
4. 테스트 파일 삭제를 통해 전체 업로드-다운로드-삭제 흐름 확인

In [5]:
# 1. 접속 설정 및 S3 클라이언트 생성
import os
import boto3
from botocore.client import Config

# 환경 변수에서 접근 정보 로드
NCLOUD_ACCESS_KEY = os.getenv("NCLOUD_ACCESS_KEY")
NCLOUD_SECRET_KEY = os.getenv("NCLOUD_SECRET_KEY")
NCLOUD_STORAGE_REGION = os.getenv("NCLOUD_STORAGE_REGION")
NCLOUD_STORAGE_BUCKET = os.getenv("NCLOUD_STORAGE_BUCKET")
NCLOUD_STORAGE_ENDPOINT_URL = os.getenv("NCLOUD_STORAGE_ENDPOINT_URL")

# S3 호환 클라이언트 생성
s3_client = boto3.client(
    "s3",
    endpoint_url=NCLOUD_STORAGE_ENDPOINT_URL,
    aws_access_key_id=NCLOUD_ACCESS_KEY,
    aws_secret_access_key=NCLOUD_SECRET_KEY,
    region_name=NCLOUD_STORAGE_REGION,
    config=Config(signature_version="s3v4"),
)

In [ ]:
# 2. DataFrame 생성 및 CSV 없로드
from io import StringIO
import pandas as pd

# 업로드 경로 설정
DATASETS_DIR = "datasets"

# 업로드 함수 정의
def upload_dataframe_as_csv(df: pd.DataFrame, index: bool = False):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=index)
    return s3_client.put_object(
        Bucket=NCLOUD_STORAGE_BUCKET,
        Key=f"{DATASETS_DIR}/weather_data.csv", 
        Body=csv_buffer.getvalue(),
    )


# 업로드 테스트
upload_dataframe_as_csv(df)

In [8]:
DATASETS_DIR = "datasets"
# S3 디렉토리 내 객체 리스트 함수
def list_s3_files_in_dir(prefix: str) -> list:
    """S3 버킷에서 특정 prefix 아래의 파일 목록을 반환합니다."""
    response = s3_client.list_objects_v2(
        Bucket=NCLOUD_STORAGE_BUCKET,
        Prefix=prefix
    )
    if "Contents" not in response:
        print("📂 해당 디렉토리에 파일이 없습니다.")
        return []
    
    file_list = [obj["Key"] for obj in response["Contents"]]
    return file_list

files = list_s3_files_in_dir(DATASETS_DIR + "/")
for f in files:
    print(f)

datasets/
datasets/20200527-20250527-100-daegwallyeong.csv
datasets/20200527-20250527-101-chuncheon.csv
datasets/20200527-20250527-102-baengnyeongdo.csv
datasets/20200527-20250527-104-north_gangneung.csv
datasets/20200527-20250527-105-gangneung.csv
datasets/20200527-20250527-106-donghae.csv
datasets/20200527-20250527-108-seoul.csv
datasets/20200527-20250527-112-incheon.csv
datasets/20200527-20250527-114-wonju.csv
datasets/20200527-20250527-115-ulleungdo.csv
datasets/20200527-20250527-119-suwon.csv
datasets/20200527-20250527-121-yeongwol.csv
datasets/20200527-20250527-127-chungju.csv
datasets/20200527-20250527-129-seosan.csv
datasets/20200527-20250527-130-uljin.csv
datasets/20200527-20250527-131-cheongju.csv
datasets/20200527-20250527-133-daejeon.csv
datasets/20200527-20250527-135-chupungnyeong.csv
datasets/20200527-20250527-136-andong.csv
datasets/20200527-20250527-137-sangju.csv
datasets/20200527-20250527-138-pohang.csv
datasets/20200527-20250527-140-gunsan.csv
datasets/20200527-20250

In [11]:
# 3. 저장된 CSV 불러오기
def read_csv_as_dataframe(key: str) -> pd.DataFrame:
    r = s3_client.get_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=key)
    file_content = r["Body"].read().decode("utf-8")
    csv_buffer = StringIO(file_content)
    return pd.read_csv(csv_buffer)

# 불러오기 테스트
read_csv_as_dataframe(f"{DATASETS_DIR}/weather_data.csv")

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,108,서울,2010-01-01,-7.6,-12.7,654,-3.6,1501,NaN,NaN,...,2.4,6.1,9.2,15.2,17.2,NaN,0.6,1.1,NaN,NaN
1,108,서울,2010-01-02,-3.6,-7.4,2359,0.2,1524,NaN,NaN,...,2.3,5.9,9.0,15.1,17.2,NaN,0.1,0.3,{눈}0158-{눈}{강도0}0300-0330. {눈}0530-{눈}{강도0}060...,0.53
2,108,서울,2010-01-03,-6.8,-10.5,748,-3.2,1438,NaN,NaN,...,2.3,5.9,9.0,15.1,17.2,NaN,1.0,8.0,NaN,NaN
3,108,서울,2010-01-04,-5.9,-8.0,2400,-3.4,1425,NaN,NaN,...,2.2,5.7,8.8,14.9,17.1,NaN,0.3,6.2,{눈}0010-0230. {눈}0428-{시정(미만)}{1km}{눈}0445-{시정...,NaN
4,108,서울,2010-01-05,-9.9,-12.3,2333,-7.0,1506,NaN,NaN,...,2.2,5.7,8.8,14.9,17.2,NaN,0.9,NaN,{눈}0553-{눈}{강도0}0600-0650.,NaN
5,108,서울,2010-01-06,-11.2,-13.3,627,-8.1,1350,NaN,NaN,...,2.2,5.7,8.6,14.8,17.1,NaN,1.1,NaN,NaN,NaN
6,108,서울,2010-01-07,-10.1,-13.6,554,-5.5,1450,NaN,NaN,...,2.1,5.6,8.5,14.7,17.1,NaN,0.7,NaN,{연무}0935-{연무}{강도0}1200-1230. {연무}2310-{연무}{강도0...,NaN
7,108,서울,2010-01-08,-8.0,-11.8,657,-3.3,1509,NaN,NaN,...,2.0,5.5,8.4,14.5,17.0,NaN,0.4,NaN,-{연무}-{연무}{강도0}0300-0421. {박무}0421-{박무}{강도0}06...,NaN
8,108,서울,2010-01-09,-5.1,-8.7,528,-1.9,1438,NaN,NaN,...,2.1,5.4,8.3,14.5,17.0,NaN,0.7,0.4,{박무}0305-{박무}{강도0}0600-{박무}{강도0}0900-0933. {연무...,NaN
9,108,서울,2010-01-10,-3.4,-5.3,507,-0.3,1418,NaN,NaN,...,2.0,5.3,8.2,14.4,17.0,NaN,2.5,NaN,-{박무}-{박무}{강도0}0300-{박무}{강도0}0600-{박무}{강도0}090...,NaN


In [12]:
# 4. 업로드된 파일 삭제
# 삭제 테스트
s3_client.delete_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=f"{DATASETS_DIR}/weather_data.csv")

{'ResponseMetadata': {'RequestId': '89e48f80-0ef5-49fd-9f97-c2bd63b5a2b0',
  'HostId': 'ODcyNGVjZjIyOWE3OWMxM2VkN2YwYWM1NTk3YTE5YTRhYTZhOTAwZjIzYjg1OWEyNDM0NmEwMDViMDUwZjcwZQ==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-delete-marker': 'true',
   'x-amz-id-2': 'ODcyNGVjZjIyOWE3OWMxM2VkN2YwYWM1NTk3YTE5YTRhYTZhOTAwZjIzYjg1OWEyNDM0NmEwMDViMDUwZjcwZQ==',
   'x-amz-request-id': '89e48f80-0ef5-49fd-9f97-c2bd63b5a2b0',
   'x-amz-version-id': 'c0c7ded7-3b73-11f0-879b-9cc2c468d0e9',
   'date': 'Wed, 28 May 2025 03:27:59 GMT',
   'server': 'Ncloud Storage'},
  'RetryAttempts': 0},
 'DeleteMarker': True,
 'VersionId': 'c0c7ded7-3b73-11f0-879b-9cc2c468d0e9'}